In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

%matplotlib inline  

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Basic data

In [3]:
''' Колонки '''
tick = '^GSPC'
ma1 = 30
ma2 = 150


''' время '''
stime='2021-01-01'
ftime=None
period = '1d'  # '1d' '1wk'

Add ticker and grow percent

In [4]:
def get_data_from_ticker(tick, stime, ftime, interval=period):
    ticker = yf.Ticker(tick)

    df = ticker.history(start=stime, end=ftime, interval=interval)
    x = pd.DataFrame(df)
    x.rename(columns={"Close": tick}, inplace=True)
    z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
    return z

df = pd.DataFrame(get_data_from_ticker(tick, stime, ftime))

In [5]:
def add_percent_update(tick):
    """ date stock's or market's update"""
    gf_copy = df.copy(deep=True)
    gf_copy = pd.DataFrame(gf_copy.drop(df.index[0]))
    gf_copy_v = gf_copy[tick].values
    gf_v = df.copy(deep=True)
    gf_vv = gf_v[tick].values

    for i in range(len(gf_copy)):
        gf_vv[i] = (gf_copy_v[i] - gf_vv[i]) / (gf_vv[i] / 100)

    gf_vv = np.insert(gf_vv, 0, 0)
    gf_vv = np.delete(gf_vv, -1)
    return gf_vv

df[f"% update {tick}"] = add_percent_update(tick)
# gf[f"% update {st}"] = add_percent_update(st)

# df = df[[f"% update {tick}", tick]]
df.head(3)

,^GSPC,% update ^GSPC
Date,,
2020-12-31,3756.070068,0.000000
2021-01-04,3700.649902,-1.475483
2021-01-05,3726.860107,0.708260


Plus and minus 

In [6]:
r = list(df[f'% update {tick}'].values)

for i in range(len(r)):
    if df.iloc[i, 1] > 0:
        r[i] = 'Plus'
    else:
        r[i] = 'Minus'
        
df['Side'] = r 

MA

In [11]:
ma1 = 12
ma2 = 26

def add_ma(df, num):
    x = df.columns[0]
    df[f'ma_{num}'] = df[x].ewm(span=num, adjust=False).mean()
    return df    

try:
    df = add_ma(df, ma1)
    df = add_ma(df, ma2)
except:
    df = add_ma(df, ma1)
finally:
    pass

# df.head(5)

Exponentially-weighted MA 

MACD

In [10]:
ma_macd_1 = 12
ma_macd_2 = 26

def add_macd(df, ma_macd_1, ma_macd_2):
    x = df.columns[0]
    df[f'ma_{ma_macd_1}'] = df[x].ewm(span=ma_macd_1, adjust=False).mean()
    df[f'ma_{ma_macd_2}'] = df[x].ewm(span=ma_macd_2, adjust=False).mean()
    df['macd'] = df[f'ma_{ma_macd_1}'] - df[f'ma_{ma_macd_2}']
    df['signal_macd'] = df['macd'].ewm(span=9, adjust=False).mean()
    df['masi'] = df['macd'] - df['signal_macd']
    return df

df = add_macd(df, ma_macd_1, ma_macd_2)

df.head(3)

,^GSPC,% update ^GSPC,Side,ma_12,ma_26,macd,signal_macd,masi
Date,,,,,,,,
2020-12-31,3756.070068,0.000000,Minus,3756.070068,3756.070068,0.000000,0.000000,0.000000
2021-01-04,3700.649902,-1.475483,Minus,3747.543889,3751.964871,-4.420982,-0.884196,-3.536786
2021-01-05,3726.860107,0.708260,Plus,3744.361769,3750.105259,-5.743490,-1.856055,-3.887435
